In [1]:
import pygame
import sys
import math
import json
import random
from PIL import Image

# Initialize Pygame
pygame.init()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [2]:
###############################
# Constants for Panels & Drawing
###############################

WIDTH, HEIGHT = 1400, 800  # Total window dimensions.
TOP_PANEL_HEIGHT = 40      # Height of the top instructions panel.

# Left GUI panel for sliders/buttons
GUI_PANEL_WIDTH = 200

# After the GUI, we have 3 panels side by side:
MAIN_PANEL_WIDTH = 400      # graph editor
SUBGRAPH_PANEL_WIDTH = 400  # subgraph viewer
THIRD_PANEL_WIDTH = 400     # composite graph

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("GrammarScape: Interactive Graph Editor")

In [3]:
###############################
# Colors & Global Parameters
###############################
BG_COLOR = (30, 30, 30)

# Graph colors
EDGE_COLOR = [200, 200, 200]
NODE_COLOR = [255, 255, 255]

# Edge effect
edge_noise = 0
edge_curve = 0

# Composite Graph generation
numIterations = 3
composite_seed = 0
composite_length_seed = 0

# 3D camera transforms
right_view_offset = [0, 0]
right_view_zoom = 1.0
right_view_yaw = 0.0
right_view_pitch = 0.0
perspective_distance = 300

right_panel_middle_dragging = False
right_panel_left_dragging = False
right_panel_right_dragging = False

right_view_last_mouse_middle = (0, 0)
right_view_last_mouse_left = (0, 0)
right_view_last_mouse_right = (0, 0)

# Grid for UI
GRID_SIZE = 20
GRID_COLOR = (50, 50, 50)

# Subgraph viewer
CELL_HEIGHT = 120
CELL_MARGIN_X = 10
CELL_MARGIN_Y = 10
GRID_LINE_COLOR = (80, 80, 80)
GRID_LINE_THICKNESS = 1

NODE_SIZE = 20
NODE_IMAGE_PATH = "node_imgs/dot.png"
node_image_main = None
try:
    tmp_img = pygame.image.load(NODE_IMAGE_PATH).convert_alpha()
    node_image_main = pygame.transform.scale(tmp_img, (NODE_SIZE, NODE_SIZE))
except:
    node_image_main = None

pygame.font.init()
font = pygame.font.SysFont(None, 18)
instr_font = pygame.font.SysFont(None, 16)

In [4]:
###############################
# Graph Class
###############################

class Graph:
    def __init__(self):
        self.nodes = []           # list of (x, y)
        self.adjacency_list = {}  # node_index -> [neighbor_indices]
        self.edge_slopes = {}     # (n1, n2) -> slope in radians

    def add_node(self, pos):
        idx = len(self.nodes)
        self.nodes.append(pos)
        self.adjacency_list[idx] = []
        return idx

    def add_edge(self, n1, n2):
        # Creates an undirected edge between nodes n1 and n2 and computes its slope
        if n2 not in self.adjacency_list[n1]:
            self.adjacency_list[n1].append(n2)
        if n1 not in self.adjacency_list[n2]:
            self.adjacency_list[n2].append(n1)
        dx = self.nodes[n2][0] - self.nodes[n1][0]
        dy = self.nodes[n2][1] - self.nodes[n1][1]
        slope = math.atan2(dy, dx)
        self.edge_slopes[(n1, n2)] = slope
        self.edge_slopes[(n2, n1)] = math.atan2(-dy, -dx)

# User-defined graph
graph = Graph()
compositeGraph = Graph()

# Composite builder
composite_nodes = []
composite_edges = []
composite_tolerance = 0.0873  # ~5 degrees
connection_length = 100
merge_threshold = 10

In [5]:
###############################
# Slider Class
###############################

class Slider:
    """
    A simple horizontal slider with:
      - (x, y, width, height) rect
      - min_val, max_val
      - value
      - is_int (bool) => round to int
      - label (string)
    """
    def __init__(self, x, y, w, h, min_val, max_val, value, label="", is_int=False):
        self.rect = pygame.Rect(x, y, w, h)
        self.min_val = min_val
        self.max_val = max_val
        self.value = value
        self.is_int = is_int
        self.label = label

        # handle radius & dragging
        self.handle_radius = h // 2
        self.dragging = False

    def draw(self, surf, mouse_pos):
        # Draw track
        track_y = self.rect.y + self.rect.height // 2
        pygame.draw.line(surf, (120,120,120),
                         (self.rect.x, track_y),
                         (self.rect.x + self.rect.width, track_y), 3)

        # Draw handle
        t = (self.value - self.min_val) / (self.max_val - self.min_val)
        handle_x = self.rect.x + int(t * self.rect.width)
        handle_y = track_y

        # If we're dragging or mouse is over the handle, highlight
        handle_color = (200, 200, 0) if self.dragging or self.handle_hit_test(mouse_pos) else (180,180,180)
        pygame.draw.circle(surf, handle_color, (handle_x, handle_y), self.handle_radius)

        # Draw label & current value
        val_text = str(int(self.value) if self.is_int else round(self.value,2))
        label_text = f"{self.label}: {val_text}"
        surf.blit(font.render(label_text, True, (255,255,255)), (self.rect.x, self.rect.y - 18))

    def handle_hit_test(self, mouse_pos):
        """Check if mouse is over the handle."""
        t = (self.value - self.min_val) / (self.max_val - self.min_val)
        handle_x = self.rect.x + int(t * self.rect.width)
        handle_y = self.rect.y + self.rect.height//2
        return math.dist(mouse_pos, (handle_x, handle_y)) <= self.handle_radius + 2

    def process_event(self, event):
        """Update `self.value` if user drags the handle."""
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if self.handle_hit_test(event.pos):
                self.dragging = True

        elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            self.dragging = False

        elif event.type == pygame.MOUSEMOTION and self.dragging:
            # convert mouse x back to slider value
            # clamp to [min_val, max_val]
            rel_x = event.pos[0] - self.rect.x
            t = rel_x / self.rect.width
            new_val = self.min_val + t*(self.max_val - self.min_val)
            # clamp
            if new_val < self.min_val:
                new_val = self.min_val
            if new_val > self.max_val:
                new_val = self.max_val
            # round if integer slider
            if self.is_int:
                new_val = int(round(new_val))
            self.value = new_val

In [6]:
###############################
# Create GUI Sliders
###############################
sliders = []

def create_sliders():
    global sliders
    sliders.clear()
    # stack them vertically in the GUI panel:
    # each slider has height=20, plus ~30 px vertical space

    left_x = 10
    top_y  = TOP_PANEL_HEIGHT + 50
    s_width = GUI_PANEL_WIDTH - 20
    s_height= 16
    gap     = 40

    # 1) Edge Color R, G, B
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 255, EDGE_COLOR[0], "Edge R", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 255, EDGE_COLOR[1], "Edge G", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 255, EDGE_COLOR[2], "Edge B", is_int=True))
    top_y += gap

    # 2) Node Color R, G, B
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 255, NODE_COLOR[0], "Node R", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 255, NODE_COLOR[1], "Node G", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 255, NODE_COLOR[2], "Node B", is_int=True))
    top_y += gap

    # 3) Noise & Curve
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 50, edge_noise, "Noise", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 50, edge_curve, "Curve", is_int=True))
    top_y += gap

    # 4) Iterations (1..10), Seeds (0..1000)
    sliders.append(Slider(left_x, top_y, s_width, s_height, 1, 10, numIterations, "Iterations", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 1000, composite_seed, "Comp Seed", is_int=True))
    top_y += gap
    sliders.append(Slider(left_x, top_y, s_width, s_height, 0, 1000, composite_length_seed, "Len Seed", is_int=True))
    top_y += gap

create_sliders()

In [7]:
###############################
# Utility Functions
###############################

def snap_to_grid(pos):
    # Snaps a given position to the nearest grid intersection
    # The graph editor used to create the example graph leverages a grid 
    # We need to do this because the composite graph extends the graph based on edge slopes. 
    x, y = pos
    return (round(x / GRID_SIZE) * GRID_SIZE, round(y / GRID_SIZE) * GRID_SIZE)

def get_node_at_pos(pos):
    for i, (nx, ny) in enumerate(graph.nodes):
        if math.dist((nx, ny), pos) < NODE_SIZE:
            return i
    return None
    
def get_edge_points(start, end, noise_intensity, curve_intensity, segments=20):
    # Computes a list of points along an edge between start and end.
    # Also applies a curve (via a control point) and random noise.
    if curve_intensity > 0:
        mid = ((start[0]+end[0])*0.5, (start[1]+end[1])*0.5)
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        length = math.hypot(dx, dy)
        perp = (-dy/length, dx/length) if length!=0 else (0,0)
        control = (mid[0] + perp[0]*curve_intensity, mid[1] + perp[1]*curve_intensity)
    else:
        control = None

    pts = []
    for i in range(segments+1):
        t = i/segments
        if control is not None:
            # Quadratic Bezier
            x = (1 - t)**2 * start[0] + 2*(1 - t)*t*control[0] + t**2*end[0]
            y = (1 - t)**2 * start[1] + 2*(1 - t)*t*control[1] + t**2*end[1]
        else:
            x = start[0] + t*(end[0]-start[0])
            y = start[1] + t*(end[1]-start[1])
        if noise_intensity>0:
            seed = hash((round(start[0],2), round(start[1],2), round(end[0],2), round(end[1],2), i))
            rng = random.Random(seed)
            x += rng.uniform(-noise_intensity, noise_intensity)
            y += rng.uniform(-noise_intensity, noise_intensity)
        pts.append((int(x), int(y)))
    return pts

def draw_jagged_or_curved_edge(surface, color, start, end, noise_intensity, curve_intensity):
    # Draws an edge from start to end with jagged/curved effects based on the given intensities
    pts = get_edge_points(start, end, noise_intensity, curve_intensity, segments=20)
    for i in range(len(pts)-1):
        pygame.draw.line(surface, color, pts[i], pts[i+1], 2)

def build_composite_graph():
    # Builds a composite graph by iteratively selecting candidate nodes and connecting them 
    # based on edges (slopes) from a primary graph. Two random generators (seeded by 
    # composite_seed and composite_length_seed) control candidate selection and edge lengths.
    
    global composite_nodes, composite_edges
    composite_nodes = []
    composite_edges = []
    compositeGraph.nodes = []
    compositeGraph.adjacency_list = {}
    compositeGraph.edge_slopes = {}

    if not graph.nodes:
        return

    # Create two random generators: one for candidate selection and one for determining edge lengths.
    rng = random.Random(composite_seed)
    rng_len = random.Random(composite_length_seed)

    # Start the composite graph using the primary node with index 0.
    # Set its position to (0,0), store the primary node index, and record available edge slopes from node 0.
    composite_nodes.append({
        "pos": (0,0),
        "primary": 0,
        "available": [
            graph.edge_slopes.get((0, nbr))
            for nbr in graph.adjacency_list[0]
            if graph.edge_slopes.get((0,nbr)) is not None
        ]
    })

    # Initialize the frontier with tuples of (node index, slope) for each available edge from the starting node.
    frontier = [(0,s) for s in composite_nodes[0]["available"]]

    # Iterate a fixed number of times to expand the composite graph.
    for _ in range(numIterations):
        new_frontier = [] # New frontier to store edges from newly added or merged nodes.

        # Process each edge in the current frontier.
        for (comp_i, slope_val) in frontier:
            # Calculate the required slope in the opposite direction (s + π), normalized to [0, 2π).
            req = (slope_val + math.pi) % (2*math.pi)
            candidates = [] # List to hold candidate primary nodes that are a good match for connection.

            # Iterate over every candidate node in the primary graph.
            for cand_id in range(len(graph.nodes)):
                half_edges = []

                # Gather all edge slopes (if defined) for the candidate node.
                for nbr in graph.adjacency_list[cand_id]:
                    sp = graph.edge_slopes.get((cand_id, nbr))
                    if sp is not None:
                        half_edges.append(sp)

                # Check each slope of the candidate node to see if it matches the required slope within a tolerance.
                for h in half_edges:
                    diff = abs((h - req + math.pi)%(2*math.pi)-math.pi)
                    if diff < composite_tolerance:
                        candidates.append((cand_id, h, half_edges))

            # If matching candidate nodes were found
            if candidates:

                # Randomly select one candidate from the list
                cand, used, half = rng.choice(candidates)

                # Get the base position from the composite node from which we're expanding
                base_pos = composite_nodes[comp_i]["pos"]

                # Randomize the connection length based on a base length and a factor between 0.5 and 1.5
                length_ = connection_length * rng_len.uniform(0.5,1.5)

                # Compute the new position using trigonometry, based on the slope 's'
                new_pos = (
                    base_pos[0] + length_*math.cos(slope_val),
                    base_pos[1] + length_*math.sin(slope_val)
                )
                
                # Determine new available edge slopes for the new node,
                # filtering out the edge used for connection.
                new_avail = [
                    x for x in half
                    if abs(((x - used + math.pi)%(2*math.pi))-math.pi) >= composite_tolerance
                ]

                # Create a candidate new node with its position, primary node, and available slopes.
                candidate_node = {
                    "pos": new_pos,
                    "primary": cand,
                    "available": new_avail
                }

                # Check if there is an existing node close enough to merge with
                merge_idx = None
                for i, ex in enumerate(composite_nodes):
                    dx = ex["pos"][0] - new_pos[0]
                    dy = ex["pos"][1] - new_pos[1]
                    dist = math.hypot(dx, dy)
                    if dist<merge_threshold:
                        merge_idx = i
                        break
                        
                if merge_idx is not None:
                    
                    # # Merge candidate_node into the existing node
                    # Unite (union) the available edges lists, avoiding duplicates
                    exist_set = set(composite_nodes[merge_idx]["available"])
                    new_set   = set(candidate_node["available"])
                    composite_nodes[merge_idx]["available"] = list(exist_set.union(new_set))

                    # Record the edge between the current node and the merged node, if not already recorded
                    e = (comp_i, merge_idx)
                    if e not in composite_edges and (e[1],e[0]) not in composite_edges:
                        composite_edges.append(e)

                    # Add new frontier entries for the merged node using its updated available slopes
                    for vv in candidate_node["available"]:
                        new_frontier.append((merge_idx, vv))
                else:
                    # No merge candidate found: add the new node as usual
                    new_idx = len(composite_nodes)
                    composite_nodes.append(candidate_node)

                    # Record an edge connecting the current node to the new node.
                    composite_edges.append((comp_i, new_idx))
                    
                    # Add each available edge from the new node to the frontier for further expansion
                    for vv in candidate_node["available"]:
                        new_frontier.append((new_idx, vv))

        # Update the frontier with the new edges for the next iteration.
        frontier = new_frontier

    # Build compositeGraph
    compositeGraph.nodes = [nd["pos"] for nd in composite_nodes]
    for i in range(len(composite_nodes)):
        compositeGraph.adjacency_list[i] = []
    for (i,j) in composite_edges:
        compositeGraph.adjacency_list[i].append(j)
        compositeGraph.adjacency_list[j].append(i)
    # compute slopes
    for i, nbrs in compositeGraph.adjacency_list.items():
        for j in nbrs:
            dx = compositeGraph.nodes[j][0] - compositeGraph.nodes[i][0]
            dy = compositeGraph.nodes[j][1] - compositeGraph.nodes[i][1]
            sp = math.atan2(dy, dx)
            compositeGraph.edge_slopes[(i,j)] = sp
            compositeGraph.edge_slopes[(j,i)] = math.atan2(-dy, -dx)

def clear_graph():
    global graph, compositeGraph
    graph = Graph()
    compositeGraph = Graph()
    build_composite_graph()

def save_adjacency_list():
    with open("adjacency_list.json", "w") as f:
        json.dump(graph.adjacency_list, f, indent=4)
    print("Saved adjacency_list.json")

def draw_button(surf, text, rect, mouse_pos):
    color = (80,80,80)
    hover = (120,120,120)
    if rect.collidepoint(mouse_pos):
        pygame.draw.rect(surf, hover, rect)
    else:
        pygame.draw.rect(surf, color, rect)
    txt_surf = font.render(text, True, (255,255,255))
    surf.blit(txt_surf, (rect.x+5, rect.y+5))

In [8]:
###############################
# Main Loop
###############################
running = True
selected_node = None

build_composite_graph()

# GUI Buttons
clear_btn_rect = pygame.Rect(10, TOP_PANEL_HEIGHT+10, 80, 30)
save_btn_rect  = pygame.Rect(100, TOP_PANEL_HEIGHT+10, 80, 30)

while running:
    screen.fill(BG_COLOR)

    # Top panel instructions
    top_rect = pygame.Rect(0,0, WIDTH, TOP_PANEL_HEIGHT)
    pygame.draw.rect(screen, (50,50,50), top_rect)
    instr_text = (
        "[Left Panel: Sliders/Buttons] "
        "Main Panel (click to add/connect). "
        "Right Panel: MMB=pan, RMB=yaw, LMB=pitch, Scroll=zoom."
    )
    screen.blit(instr_font.render(instr_text, True, (230,230,230)), (10,10))

    # Draw the three main panels
    # 1) Main panel for user graph
    main_panel_rect = pygame.Rect(GUI_PANEL_WIDTH, TOP_PANEL_HEIGHT,
                                  MAIN_PANEL_WIDTH, HEIGHT - TOP_PANEL_HEIGHT)
    pygame.draw.rect(screen, BG_COLOR, main_panel_rect)

    # 2) Subgraph panel
    subgraph_panel_rect = pygame.Rect(GUI_PANEL_WIDTH+MAIN_PANEL_WIDTH,
                                      TOP_PANEL_HEIGHT,
                                      SUBGRAPH_PANEL_WIDTH,
                                      HEIGHT - TOP_PANEL_HEIGHT)
    pygame.draw.rect(screen, BG_COLOR, subgraph_panel_rect)

    # 3) Composite panel
    right_panel_rect = pygame.Rect(GUI_PANEL_WIDTH+MAIN_PANEL_WIDTH+SUBGRAPH_PANEL_WIDTH,
                                   TOP_PANEL_HEIGHT,
                                   THIRD_PANEL_WIDTH,
                                   HEIGHT-TOP_PANEL_HEIGHT)
    pygame.draw.rect(screen, BG_COLOR, right_panel_rect)
    pygame.draw.rect(screen, (200,200,200), right_panel_rect, 2)

    mouse_pos = pygame.mouse.get_pos()
    mouse_click = False

    # ==============================
    # Draw the GUI Panel (Left side)
    # ==============================
    gui_rect = pygame.Rect(0, TOP_PANEL_HEIGHT, GUI_PANEL_WIDTH, HEIGHT - TOP_PANEL_HEIGHT)
    pygame.draw.rect(screen, (40,40,40), gui_rect)
    pygame.draw.rect(screen, (80,80,80), gui_rect, 2)

    # Buttons
    draw_button(screen, "Clear", clear_btn_rect, mouse_pos)
    draw_button(screen, "Save", save_btn_rect, mouse_pos)

    # Draw Sliders
    for slider in sliders:
        slider.draw(screen, mouse_pos)

    # ==============================
    # Main Panel: Graph
    # ==============================
    main_off_x = GUI_PANEL_WIDTH
    main_off_y = TOP_PANEL_HEIGHT

    # Draw grid
    for gx in range(main_off_x, main_off_x+MAIN_PANEL_WIDTH, GRID_SIZE):
        pygame.draw.line(screen, GRID_COLOR, (gx,TOP_PANEL_HEIGHT), (gx,HEIGHT), 1)
    for gy in range(main_off_y, HEIGHT, GRID_SIZE):
        pygame.draw.line(screen, GRID_COLOR, (main_off_x,gy), (main_off_x+MAIN_PANEL_WIDTH,gy), 1)

    # Draw edges
    for n1 in graph.adjacency_list:
        for n2 in graph.adjacency_list[n1]:
            st = (graph.nodes[n1][0] + main_off_x, graph.nodes[n1][1] + main_off_y)
            en = (graph.nodes[n2][0] + main_off_x, graph.nodes[n2][1] + main_off_y)
            draw_jagged_or_curved_edge(screen, EDGE_COLOR, st, en, edge_noise, edge_curve)

    # Draw nodes
    for i,(nx,ny) in enumerate(graph.nodes):
        sx = nx + main_off_x
        sy = ny + main_off_y
        if node_image_main:
            screen.blit(node_image_main, (sx - NODE_SIZE//2, sy - NODE_SIZE//2))
        else:
            pygame.draw.circle(screen, NODE_COLOR, (sx,sy), NODE_SIZE//2)
            pygame.draw.circle(screen, (0,0,0), (sx,sy), NODE_SIZE//2,1)
        lbl_sf = font.render(str(i), True, (255,255,255))
        screen.blit(lbl_sf, (sx - lbl_sf.get_width()//2, sy - NODE_SIZE//2 - lbl_sf.get_height() - 2))

    # ==============================
    # Subgraph Panel
    # ==============================
    sub_off_x = GUI_PANEL_WIDTH + MAIN_PANEL_WIDTH
    sub_off_y = TOP_PANEL_HEIGHT

    num_nodes = len(graph.nodes)
    if num_nodes>0:
        base_w = 120
        gpr = max(1, int(SUBGRAPH_PANEL_WIDTH // base_w))
        dyn_w = SUBGRAPH_PANEL_WIDTH / gpr
        rows = math.ceil(num_nodes / gpr)

        for i in range(num_nodes):
            row = i // gpr
            col = i % gpr
            sub_nodes = [i] + graph.adjacency_list[i]
            sub_nodes = list(set(sub_nodes))
            xs_ = [graph.nodes[n][0] for n in sub_nodes]
            ys_ = [graph.nodes[n][1] for n in sub_nodes]
            minx, maxx = min(xs_), max(xs_)
            miny, maxy = min(ys_), max(ys_)
            w = maxx-minx or 1
            h = maxy-miny or 1
            usable_w = dyn_w - 2*CELL_MARGIN_X
            usable_h = CELL_HEIGHT - 2*CELL_MARGIN_Y
            scale = min(usable_w/w, usable_h/h)
            scale = min(scale,1.0)
            cell_left = sub_off_x + col*dyn_w
            cell_top  = sub_off_y + row*CELL_HEIGHT
            offx = cell_left + CELL_MARGIN_X
            offy = cell_top  + CELL_MARGIN_Y

            for n2 in graph.adjacency_list[i]:
                x1,y1 = graph.nodes[i]
                x2,y2 = graph.nodes[n2]
                mx1 = offx + int((x1 - minx)*scale)
                my1 = offy + int((y1 - miny)*scale)
                mx2 = offx + int((x2 - minx)*scale)
                my2 = offy + int((y2 - miny)*scale)
                pygame.draw.line(screen, (180,180,180), (mx1,my1),(mx2,my2),1)

            for n_idx in sub_nodes:
                nx_, ny_ = graph.nodes[n_idx]
                mx = offx + int((nx_ - minx)*scale)
                my = offy + int((ny_ - miny)*scale)
                mini_node_size = max(2, (NODE_SIZE//2)*scale)
                color_ = NODE_COLOR if n_idx==i else (200,200,200)
                pygame.draw.circle(screen, color_, (mx,my), mini_node_size)
                pygame.draw.circle(screen, (0,0,0), (mx,my), mini_node_size,1)
                if n_idx==i:
                    label_sf = font.render(str(n_idx), True, (255,255,255))
                    screen.blit(label_sf, (mx - label_sf.get_width()//2,
                                           my - mini_node_size - label_sf.get_height() - 2))

        # subgraph grid lines
        for c in range(gpr+1):
            gx = sub_off_x + c*dyn_w
            gy1= sub_off_y
            gy2= min(rows*CELL_HEIGHT+sub_off_y, HEIGHT)
            pygame.draw.line(screen, GRID_LINE_COLOR, (gx,gy1),(gx,gy2), GRID_LINE_THICKNESS)
        for r in range(rows+1):
            gy = sub_off_y + r*CELL_HEIGHT
            gx1= sub_off_x
            gx2= sub_off_x + gpr*dyn_w
            if gx2> sub_off_x + SUBGRAPH_PANEL_WIDTH:
                gx2= sub_off_x + SUBGRAPH_PANEL_WIDTH
            pygame.draw.line(screen, GRID_LINE_COLOR, (gx1,gy),(gx2,gy), GRID_LINE_THICKNESS)

    # ==============================
    # Right (Composite) Panel
    # ==============================
    comp_off_x = GUI_PANEL_WIDTH + MAIN_PANEL_WIDTH + SUBGRAPH_PANEL_WIDTH
    comp_off_y = TOP_PANEL_HEIGHT
    comp_w = THIRD_PANEL_WIDTH
    comp_h = HEIGHT - TOP_PANEL_HEIGHT

    if composite_nodes:
        xs = [n["pos"][0] for n in composite_nodes]
        ys = [n["pos"][1] for n in composite_nodes]
        minx, maxx = min(xs), max(xs)
        miny, maxy = min(ys), max(ys)
        bw = maxx-minx or 1
        bh = maxy-miny or 1
        base_scale = min((comp_w-20)/bw, (comp_h-20)/bh)
        scale = base_scale * right_view_zoom
        offset_x = comp_off_x + (comp_w - bw*scale)*0.5 - minx*scale + right_view_offset[0]
        offset_y = comp_off_y + (comp_h - bh*scale)*0.5 - miny*scale + right_view_offset[1]

        transformed = []
        for node in composite_nodes:
            x_, y_ = node["pos"]
            # yaw
            x_yaw = x_*math.cos(right_view_yaw) - 0*math.sin(right_view_yaw)
            z_yaw = x_*math.sin(right_view_yaw) + 0*math.cos(right_view_yaw)
            # pitch
            y_pitch = y_*math.cos(right_view_pitch) - z_yaw*math.sin(right_view_pitch)
            z_pitch = y_*math.sin(right_view_pitch) + z_yaw*math.cos(right_view_pitch)
            # perspective
            pf = perspective_distance/(perspective_distance - z_pitch)
            x_eff = x_yaw*pf
            y_eff = y_pitch*pf
            transformed.append((x_eff, y_eff))

        # edges
        for (i,j) in composite_edges:
            p1 = transformed[i]
            p2 = transformed[j]
            st = (int(p1[0]*scale + offset_x), int(p1[1]*scale + offset_y))
            en = (int(p2[0]*scale + offset_x), int(p2[1]*scale + offset_y))
            draw_jagged_or_curved_edge(screen, (0,255,0), st, en, edge_noise, edge_curve)
        # nodes
        for i, (xx,yy) in enumerate(transformed):
            sx = int(xx*scale + offset_x)
            sy = int(yy*scale + offset_y)
            pygame.draw.circle(screen, (255,0,0), (sx,sy), 5)
            lbl2 = font.render(str(i), True, (255,255,255))
            screen.blit(lbl2, (sx-lbl2.get_width()//2, sy-15))

    # =================================
    # Event Handling
    # =================================
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Let each slider see if it needs to update
        for slider in sliders:
            slider.process_event(event)

        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                mouse_click = True
                # Check if user clicked in right panel => might start pitch drag
                if right_panel_rect.collidepoint(event.pos):
                    right_panel_left_dragging = True
                    right_view_last_mouse_left = event.pos
                # Check if user clicked in main panel => add/connect nodes
                elif main_panel_rect.collidepoint(event.pos):
                    adj_pos = (event.pos[0]-main_off_x, event.pos[1]-main_off_y)
                    n_idx = get_node_at_pos(adj_pos)
                    if n_idx is None:
                        # snap to grid or not? If you want:
                        adj_pos = snap_to_grid(adj_pos)
                        graph.add_node(adj_pos)
                    else:
                        selected_node = n_idx

            if event.button == 2:  # MMB
                if right_panel_rect.collidepoint(event.pos):
                    right_panel_middle_dragging = True
                    right_view_last_mouse_middle = event.pos
            if event.button == 3:  # RMB
                if right_panel_rect.collidepoint(event.pos):
                    right_panel_right_dragging = True
                    right_view_last_mouse_right = event.pos

        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                right_panel_left_dragging = False
                # Possibly finishing an edge in main panel
                if main_panel_rect.collidepoint(event.pos) and selected_node is not None:
                    adj_pos = (event.pos[0]-main_off_x, event.pos[1]-main_off_y)
                    tgt = get_node_at_pos(adj_pos)
                    if tgt is not None and tgt!=selected_node:
                        graph.add_edge(selected_node, tgt)
                        build_composite_graph()
                    selected_node = None
            elif event.button == 2:
                right_panel_middle_dragging = False
            elif event.button == 3:
                right_panel_right_dragging = False

        elif event.type == pygame.MOUSEMOTION:
            if right_panel_middle_dragging:
                dx = event.pos[0] - right_view_last_mouse_middle[0]
                dy = event.pos[1] - right_view_last_mouse_middle[1]
                right_view_offset[0]+=dx
                right_view_offset[1]+=dy
                right_view_last_mouse_middle=event.pos
            if right_panel_left_dragging:
                dy = event.pos[1] - right_view_last_mouse_left[1]
                right_view_pitch += dy * 0.005
                right_view_last_mouse_left=event.pos
            if right_panel_right_dragging:
                dx = event.pos[0] - right_view_last_mouse_right[0]
                right_view_yaw += dx * 0.005
                right_view_last_mouse_right=event.pos

        elif event.type == pygame.MOUSEWHEEL:
            right_view_zoom *= (1 + event.y*0.1)

    # If user clicked, check the two buttons
    if mouse_click:
        if clear_btn_rect.collidepoint(mouse_pos):
            clear_graph()
            build_composite_graph()
        if save_btn_rect.collidepoint(mouse_pos):
            save_adjacency_list()

    # =================================
    # Sync Slider Values back to Globals
    # =================================
    # (We do this after we process_event, so the slider's .value is fresh)
    # Edge color
    EDGE_COLOR[0] = sliders[0].value
    EDGE_COLOR[1] = sliders[1].value
    EDGE_COLOR[2] = sliders[2].value
    
    # Node color
    NODE_COLOR[0] = sliders[3].value
    NODE_COLOR[1] = sliders[4].value
    NODE_COLOR[2] = sliders[5].value

    # Noise / Curve
    globals()["edge_noise"] = sliders[6].value
    globals()["edge_curve"] = sliders[7].value

    # Iterations / Seeds
    new_iter = sliders[8].value
    new_seed = sliders[9].value
    new_len_seed = sliders[10].value

    # If they've changed, rebuild
    changed = False
    if new_iter != numIterations:
        globals()["numIterations"] = new_iter
        changed = True
    if new_seed != composite_seed:
        globals()["composite_seed"] = new_seed
        changed = True
    if new_len_seed != composite_length_seed:
        globals()["composite_length_seed"] = new_len_seed
        changed = True

    if changed:
        build_composite_graph()

    pygame.display.flip()

pygame.quit()
sys.exit()

SystemExit: 

C:\Users\littl\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
